In [42]:
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import locale
from locale import atof
from scipy import stats

In [32]:
# Load in csv
filepath = os.path.join("CSV Files\\TWFC CSV\\Employment.csv")
employment_numbers_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\\TWFC CSV\\Un_numbers.csv")
unemployment_numbers_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\\TWFC CSV\\Un_rate.csv")
unemployment_rate_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\TWFC CSV\\Laborforce.csv")
laborforce_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\\TWFC CSV\\Income.csv")
income_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\\TWFC CSV\\Population.csv")
population_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\\Zillow CSV\\Price.csv")
price_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\\Zillow CSV\\Rent.csv")
rent_df = pd.read_csv(filepath)

In [33]:
# Merge the monthly CSVs
merge_df = pd.merge(employment_numbers_df, unemployment_numbers_df, how="outer", on=["Year", "Period ID", "Period", "Area"])
merge_df2 = pd.merge(merge_df, unemployment_rate_df, how="outer", on=["Year", "Period ID", "Period", "Area"])
merge_df3 = pd.merge(merge_df2, laborforce_df, how="outer", on=["Year", "Period ID", "Period", "Area"])

In [34]:
# Select desired columns 
merge_df = merge_df3.drop(merge_df3.columns[[2,4,6,8,10]], axis=1)

# Change column name of Area to City
merge_df=merge_df.rename(columns = {'Area':'City', 'Period ID': 'Month'})

# Slice out MSA out of the City Names
merge_df["City"] = merge_df["City"].replace(' MSA','',regex=True)
merge_df = merge_df

# Convert Strings to Floats
merge_df["Employment"] = merge_df["Employment"].replace(',','',regex=True).astype('float')
merge_df["Unemployment"] = merge_df["Unemployment"].replace(',','',regex=True).astype('float')
merge_df["Labor Force"] = merge_df["Labor Force"].replace(',','',regex=True).astype('float')

#Clean up city names
merge_df['City'], merge_df['x'] = merge_df["City"].str.split('-', 2).str[0:2].str
merge_df = merge_df.drop(merge_df.columns[[-1]], axis=1)

In [35]:
# Convert Region Name into Seperate City and State Columns
rent_df['City'],rent_df['State'] = rent_df["RegionName"].str.split(',', 2).str[0:2].str
rent_df['State'].replace(' TX','TX', inplace=True)
price_df['City'],price_df['State'] = price_df["RegionName"].str.split(',', 2).str[0:2].str
price_df['State'].replace(' TX','TX', inplace=True)

# Extract only the Texas Data
rent_tx_df = rent_df.loc[rent_df["State"] == "TX", :]
rent_tx_df = rent_tx_df.set_index('City')
price_tx_df = price_df.loc[price_df["State"] == "TX", :]
price_tx_df = price_tx_df.set_index('City')

# Remove Undesired Columns
rent_tx_df = rent_tx_df.drop(rent_tx_df.columns[[0,1,2,-1]], axis=1)
price_tx_df = price_tx_df.drop(price_tx_df.columns[[0,1,-1]], axis=1)

# Transpose Data Frame
rent = rent_tx_df.transpose()
price = price_tx_df.transpose()

#Move the Index
rent.reset_index(level=0, inplace=True)
rent = rent.rename(columns = {'index':'Date'})
rent = rent.reset_index(drop=True)
price.reset_index(level=0, inplace=True)
price = price.rename(columns = {'index':'Date'})
price = price.reset_index(drop=True)

# Convert the Date into a Month and Year Column
rent['Year'],rent['Month'] = rent["Date"].str.split('-', 2).str[0:2].str
rent = rent.drop(rent.columns[[0]], axis=1)
price['Year'],price['Month'] = price["Date"].str.split('-', 2).str[0:2].str
price = price.drop(price.columns[[0]], axis=1)

# Melt Data Frames and Restructure and Merge
rent_melt = pd.melt(rent, id_vars=["Year", "Month"])
rent_melt = rent_melt.rename(columns={'value': 'Rent'})
rent_melt = rent_melt.replace('Dallas-Fort Worth', 'Dallas')
price_melt = pd.melt(price, id_vars=["Year", "Month"])
price_melt = price_melt.rename(columns={'value': 'House Price'})
merge_melts = pd.merge(rent_melt,price_melt, how='left', on=['Year','Month','City'])

In [36]:
# Merge TWFC Data with Zillow Data
monthly_merge = pd.merge(merge_df, merge_melts, how="outer", on=["Year", "Month", "City"])

monthly_merge.head()

,Year,Month,City,Employment,Unemployment,Unemployment Rate,Labor Force,Rent,House Price
0,2017.0,1.0,Abilene,71281.0,3044.0,4.1,74325.0,NaN,NaN
1,2017.0,2.0,Abilene,71645.0,3224.0,4.3,74869.0,NaN,NaN
2,2017.0,3.0,Abilene,71935.0,3214.0,4.3,75149.0,NaN,NaN
3,2017.0,4.0,Abilene,72095.0,3155.0,4.2,75250.0,NaN,NaN
4,2017.0,5.0,Abilene,71677.0,3094.0,4.1,74771.0,NaN,NaN


In [37]:
# Clean up Income data to merge to annual data frame
# Select desired columns 
income_df = income_df.drop(income_df.columns[[1,3,4]], axis=1)

# Change column name of Area to City
income_df = income_df.rename(columns = {'Area':'City'})

# Slice out MSA out of the City Names
income_df["City"] = income_df["City"].replace(' MSA','',regex=True)

#Clean up city names
income_df['City'], income_df['x'] = income_df["City"].str.split('-', 2).str[0:2].str
income_df = income_df.drop(income_df.columns[[-1]], axis=1)

# Convert Strings to Floats
income_df["Income"] = income_df["Income"].replace(',','',regex=True)

income = []
for x in income_df["Income"]:
    x = x[1:]
    x = int(x)
    income.append(x)

income_df["Income"] = income

# Merge Income Data Frame to the Main Data Frame
df_final = pd.merge(monthly_merge, income_df, how="outer", on=["City", "Year"])
df_final['City'].unique()

array(['Abilene', 'Amarillo', 'Austin', 'Beaumont', 'Brownsville',
       'College Station', 'Corpus Christi', 'Dallas', 'El Paso', 'Houston',
       'Killeen', 'Laredo', 'Longview', 'Lubbock', 'McAllen', 'Midland',
       'Odessa', 'San Angelo', 'San Antonio', 'Sherman', 'Texarkana',
       'Tyler', 'Victoria', 'Waco', 'Wichita Falls', 'Huntsville',
       'Marshall', 'Palestine', 'Eagle Pass', 'Jacksonville', 'Kerrville',
       'Del Rio', 'Corsicana', 'El Campo', 'Alice', 'Gainesville',
       'Brownwood', 'Stephenville', 'Plainview', 'Big Spring',
       'Sulphur Springs', 'Brenham', 'Kingsville', 'Fredericksburg',
       'Pampa', 'Dumas', 'Snyder', 'Sweetwater', 'Andrews', 'Pecos',
       'Vernon'], dtype=object)

In [ ]:
df_final = df_final.groupby(['Year','City']).mean()

In [41]:
Y2010 =df_final.loc[2010]
Y2010 = Y2010.dropna()
Y2010

,Month,Employment,Unemployment,Unemployment Rate,Labor Force,Rent,House Price,Income
City,,,,,,,,
Abilene,7.214286,7.112175e+04,5326.750000,6.958333,7.644850e+04,994.0,125600.0,34093.0
Amarillo,7.214286,1.222838e+05,7352.500000,5.675000,1.296363e+05,1109.0,139900.0,37003.0
Austin,7.214286,8.655941e+05,65048.250000,6.991667,9.306423e+05,1312.0,208002.5,40078.0
Beaumont,7.214286,1.628125e+05,20656.750000,11.258333,1.834692e+05,1133.5,135875.0,34387.0
Brownsville,7.214286,1.441722e+05,18127.916667,11.175000,1.623002e+05,938.0,179950.0,23152.0
Corpus Christi,7.214286,1.853486e+05,17240.916667,8.500000,2.025895e+05,1159.5,158300.0,35586.0
Dallas,7.214286,3.034311e+06,266387.583333,8.075000,3.300698e+06,1297.0,162450.0,40613.0
El Paso,7.214286,3.109430e+05,31300.916667,9.133333,3.422439e+05,1087.5,141000.0,28074.0
Houston,7.214286,2.724919e+06,245356.250000,8.258333,2.970275e+06,1365.0,169445.0,44498.0


In [48]:
stats.pearsonr(Y2010['Income'], Y2010['Rent'])

(0.71555196431565393, 0.0008413058896690899)